<a href="https://colab.research.google.com/github/nilnida/DSA210-Term-Project/blob/main/data_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Processing

In [9]:
from google.colab import drive
drive.mount('/content/drive')

import scipy.io
import numpy as np
import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Charging, Discharging and Impedance Data over Cycles

This data has been taken from a Matlab file exported from my Drive folder and seperated based on cycle types. Then, each data frame has been saved to corresponding csv files in the same folder that they were taken.

In [28]:
file_path1 = "/content/drive/MyDrive/dsa210/battery_dataset/B0005.mat"

mat_data = scipy.io.loadmat(file_path1)
b0005 = mat_data['B0005']
cycle_struct = b0005[0, 0]['cycle']

charge_records_B0005 = []
discharge_records_B0005 = []
impedance_records_B0005 = []

for i in range(cycle_struct.shape[1]):
  cycle = cycle_struct[0, i]
  cycle_type = cycle['type'][0]
  amb_temp = cycle['ambient_temperature'].flatten()[0]
  data = cycle['data']

  if cycle_type == "charge":
    voltage = data['Voltage_measured'][0, 0].flatten()
    current = data['Current_measured'][0, 0].flatten()
    temperature = data['Temperature_measured'][0, 0].flatten()
    current_charge = data['Current_charge'][0, 0].flatten()
    voltage_charge = data['Voltage_charge'][0, 0].flatten()
    time = data['Time'][0, 0].flatten()

    for j in range(len(voltage)):
      charge_records_B0005.append({
        "Cycle index": i+1,
        "Ambient temperature (°C)": amb_temp,
        "Time (secs)": time[j],
        "Battery terminal voltage (Volts)": voltage[j],
        "Battery output current (Amps)": current[j],
        "Battery temperature (°C)": temperature[j],
        "Current measured at charger (Amps)": current_charge[j],
        "Voltage measured at charger (Volts)": voltage_charge[j]
      })

  elif cycle_type == "discharge":
    voltage = data['Voltage_measured'][0, 0].flatten()
    current = data['Current_measured'][0, 0].flatten()
    temperature = data['Temperature_measured'][0, 0].flatten()
    current_load = data['Current_load'][0, 0].flatten()
    voltage_load = data['Voltage_load'][0, 0].flatten()
    time = data['Time'][0, 0].flatten()
    capacity = data['Capacity'][0, 0].flatten()[0]

    for j in range(len(voltage)):
      discharge_records_B0005.append({
        "Cycle index": i+1,
        "Ambient temperature (°C)": amb_temp,
        "Battery terminal voltage (Volts)": voltage[j],
        "Battery output current (Amps)": current[j],
        "Battery temperature (°C)": temperature[j],
        "Current measured at load (Amps)": current_load[j],
        "Voltage measured at load (Volts)": voltage_load[j],
        "Time (secs)": time[j],
        "Battery capacity (Ahr) for discharge till 2.7V": capacity
      })

  elif cycle_type == "impedance":
    sense_current = data['Sense_current'][0, 0].flatten()
    battery_current = data['Battery_current'][0, 0].flatten()
    current_ratio = data['Current_ratio'][0, 0].flatten()
    battery_impedance = data['Battery_impedance'][0, 0].flatten()
    rectified_impedance = data['Rectified_Impedance'][0, 0].flatten()
    Re = data['Re'][0, 0].flatten()[0]
    Rct = data['Rct'][0, 0].flatten()[0]

    for j in range(len(sense_current)):
      impedance_records_B0005.append({
        "Cycle index": i+1,
        "Ambient temperature (°C)": amb_temp,
        "Current in sense branch (Amps)": sense_current[j],
        "Current in battery branch (Amps)": battery_current[j],
        "Ratio of the above currents": current_ratio[j],
        "Battery impedance (Ohms) computed from raw data": battery_impedance[j],
        "Calibrated and smoothed battery impedance (Ohms)": rectified_impedance[j] if j < len(rectified_impedance) else None,
        "Estimated electrolyte resistance (Ohms)": Re,
        "Estimated charge transfer resistance (Ohms)": Rct
      })

df_charge_B0005 = pd.DataFrame(charge_records_B0005)
df_discharge_B0005 = pd.DataFrame(discharge_records_B0005)
df_impedance_B0005 = pd.DataFrame(impedance_records_B0005)

charge_output_file = "/content/drive/MyDrive/dsa210/df_charge_B0005.csv"
discharge_output_file = "/content/drive/MyDrive/dsa210/df_discharge_B0005.csv"
impedance_output_file = "/content/drive/MyDrive/dsa210/df_impedance_B0005.csv"

df_charge_B0005.to_csv(charge_output_file, index=False)
print(f"Data has been saved to {charge_output_file}")

df_discharge_B0005.to_csv(discharge_output_file, index=False)
print(f"Data has been saved to {discharge_output_file}")

df_impedance_B0005.to_csv(impedance_output_file, index=False)
print(f"Data has been saved to {impedance_output_file}")

Data has been saved to /content/drive/MyDrive/dsa210/df_charge_B0005.csv
Data has been saved to /content/drive/MyDrive/dsa210/df_discharge_B0005.csv
Data has been saved to /content/drive/MyDrive/dsa210/df_impedance_B0005.csv


In [30]:
print("Charge data sample:")
display(df_charge_B0005.head())
display(df_charge_B0005.tail())

Charge data sample:


,Cycle index,Ambient temperature (°C),Time (secs),Battery terminal voltage (Volts),Battery output current (Amps),Battery temperature (°C),Current measured at charger (Amps),Voltage measured at charger (Volts)
0,1,24,0.000,3.873017,-0.001201,24.655358,0.000,0.003
1,1,24,2.532,3.479394,-4.030268,24.666480,-4.036,1.570
2,1,24,5.500,4.000588,1.512731,24.675394,1.500,4.726
3,1,24,8.344,4.012395,1.509063,24.693865,1.500,4.742
4,1,24,11.125,4.019708,1.511318,24.705069,1.500,4.753


,Cycle index,Ambient temperature (°C),Time (secs),Battery terminal voltage (Volts),Battery output current (Amps),Battery temperature (°C),Current measured at charger (Amps),Voltage measured at charger (Volts)
541168,616,24,0.000,0.236356,-0.003484,23.372048,0.000,0.003
541169,616,24,2.547,0.003365,-0.001496,23.369434,0.000,0.003
541170,616,24,5.500,4.985137,0.000506,23.386535,0.000,5.002
541171,616,24,8.312,4.984720,0.000442,23.386983,-0.002,5.002
541172,616,24,12.656,4.213440,-0.000734,23.385061,-0.002,4.229


In [31]:
print("Discharge data sample:")
display(df_discharge_B0005.head())
display(df_discharge_B0005.tail())

Discharge data sample:


,Cycle index,Ambient temperature (°C),Battery terminal voltage (Volts),Battery output current (Amps),Battery temperature (°C),Current measured at load (Amps),Voltage measured at load (Volts),Time (secs),Battery capacity (Ahr) for discharge till 2.7V
0,2,24,4.191492,-0.004902,24.330034,-0.0006,0.000,0.000,1.856487
1,2,24,4.190749,-0.001478,24.325993,-0.0006,4.206,16.781,1.856487
2,2,24,3.974871,-2.012528,24.389085,-1.9982,3.062,35.703,1.856487
3,2,24,3.951717,-2.013979,24.544752,-1.9982,3.030,53.781,1.856487
4,2,24,3.934352,-2.011144,24.731385,-1.9982,3.011,71.922,1.856487


,Cycle index,Ambient temperature (°C),Battery terminal voltage (Volts),Battery output current (Amps),Battery temperature (°C),Current measured at load (Amps),Voltage measured at load (Volts),Time (secs),Battery capacity (Ahr) for discharge till 2.7V
50280,614,24,3.579262,-0.001569,34.864823,0.0006,0.0,2781.312,1.325079
50281,614,24,3.581964,-0.003067,34.814770,0.0006,0.0,2791.062,1.325079
50282,614,24,3.584484,-0.003079,34.676258,0.0006,0.0,2800.828,1.325079
50283,614,24,3.587336,0.001219,34.565580,0.0006,0.0,2810.640,1.325079
50284,614,24,3.589937,-0.000583,34.405920,0.0006,0.0,2820.390,1.325079


In [32]:
print("Impedance data sample:")
display(df_impedance_B0005.head())
display(df_impedance_B0005.tail())

Impedance data sample:


,Cycle index,Ambient temperature (°C),Current in sense branch (Amps),Current in battery branch (Amps),Ratio of the above currents,Battery impedance (Ohms) computed from raw data,Calibrated and smoothed battery impedance (Ohms),Estimated electrolyte resistance (Ohms),Estimated charge transfer resistance (Ohms)
0,41,24,-1.000000+ 1.000000j,-1.000000+ 1.000000j,1.000000+0.000000j,-0.438926-0.107298j,0.070069-0.000480j,0.044669,0.069456
1,41,24,820.609497- 36.234550j,337.091461- 82.920776j,2.320415+0.463305j,0.130088-0.197115j,0.068179-0.001190j,0.044669,0.069456
2,41,24,827.242188- 48.231228j,330.631561- 70.013718j,2.424193+0.367465j,0.058771+0.033307j,0.067933-0.000057j,0.044669,0.069456
3,41,24,827.193481- 56.195717j,330.808624- 61.734425j,2.447002+0.286778j,0.005814-0.060547j,0.066918-0.000879j,0.044669,0.069456
4,41,24,824.929504- 53.241478j,332.682678- 57.629013j,2.434305+0.261646j,0.126081-0.090444j,0.068071-0.000197j,0.044669,0.069456


,Cycle index,Ambient temperature (°C),Current in sense branch (Amps),Current in battery branch (Amps),Ratio of the above currents,Battery impedance (Ohms) computed from raw data,Calibrated and smoothed battery impedance (Ohms),Estimated electrolyte resistance (Ohms),Estimated charge transfer resistance (Ohms)
13339,615,24,915.489014- 64.045120j,230.149506+ 91.909889j,3.334835-1.610038j,0.245024-0.049836j,NaN+0.0j,0.050036,0.074792
13340,615,24,916.725525+ 2.986217j,212.188858+107.745811j,3.440393-1.732898j,0.264594-0.055235j,NaN+0.0j,0.050036,0.074792
13341,615,24,914.619629+126.111481j,176.598038+131.682785j,3.670656-2.022960j,0.288571-0.058837j,NaN+0.0j,0.050036,0.074792
13342,615,24,880.340820+293.825287j,136.847626+146.881027j,4.060164-2.210749j,0.317700-0.059127j,NaN+0.0j,0.050036,0.074792
13343,615,24,801.361816+450.083099j,97.058853+150.046997j,4.550338-2.397324j,0.352680-0.053818j,NaN+0.0j,0.050036,0.074792


## Remaining Useful Life Data over Cycles

In [34]:
file_path2 = '/content/drive/MyDrive/dsa210/battery_RUL.csv'
df = pd.read_csv(file_path2)

display(df.head())
display(df.tail())

,Cycle_Index,Discharge Time (s),Decrement 3.6-3.4V (s),Max. Voltage Dischar. (V),Min. Voltage Charg. (V),Time at 4.15V (s),Time constant current (s),Charging time (s),RUL
0,1.0,2595.30,1151.488500,3.670,3.211,5460.001,6755.01,10777.82,1112
1,2.0,7408.64,1172.512500,4.246,3.220,5508.992,6762.02,10500.35,1111
2,3.0,7393.76,1112.992000,4.249,3.224,5508.993,6762.02,10420.38,1110
3,4.0,7385.50,1080.320667,4.250,3.225,5502.016,6762.02,10322.81,1109
4,6.0,65022.75,29813.487000,4.290,3.398,5480.992,53213.54,56699.65,1107


,Cycle_Index,Discharge Time (s),Decrement 3.6-3.4V (s),Max. Voltage Dischar. (V),Min. Voltage Charg. (V),Time at 4.15V (s),Time constant current (s),Charging time (s),RUL
15059,1108.0,770.44,179.523810,3.773,3.742,922.775,1412.38,6678.88,4
15060,1109.0,771.12,179.523810,3.773,3.744,915.512,1412.31,6670.38,3
15061,1110.0,769.12,179.357143,3.773,3.742,915.513,1412.31,6637.12,2
15062,1111.0,773.88,162.374667,3.763,3.839,539.375,1148.00,7660.62,1
15063,1112.0,677537.27,142740.640000,4.206,3.305,49680.004,599830.14,599830.14,0


## Impedance Data over State of Charge of the Battery

In [36]:
file_path3 = '/content/drive/MyDrive/dsa210/impedance.csv'
df = pd.read_csv(file_path3)

display(df.head())
display(df.tail())

,MEASURE_ID,SOC,BATTERY_ID,FREQUENCY_ID,IMPEDANCE_VALUE
0,02_4,100,2,0,(0.110973570048518-0.00547305228273139j)
1,02_4,90,2,0,(0.10776687163617-0.00597670185700764j)
2,02_4,80,2,0,(0.107165211026746-0.0063320957096541j)
3,02_4,70,2,0,(0.106558961363241-0.00648856565733705j)
4,02_4,60,2,0,(0.106086309875063-0.00559046370050435j)


,MEASURE_ID,SOC,BATTERY_ID,FREQUENCY_ID,IMPEDANCE_VALUE
3355,06_8,50,6,13,(0.0831827899695344-0.00307402152638018j)
3356,06_8,40,6,13,(0.0836853628948842-0.00309192918947813j)
3357,06_8,30,6,13,(0.083958663235773-0.00311516608387823j)
3358,06_8,20,6,13,(0.0847027877595139-0.00332949067844146j)
3359,06_8,10,6,13,(0.0857186306576625-0.00344430936402169j)
